In [4]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import os
from tqdm.notebook import tqdm
from data_processing.GameFeaturesGenerator import GameFeaturesGenerator
from data_processing.TournamentDataProcessor import TournamentDataProcessor
from data_processing.LeaguesDataProcessor import LeaguesDataProcessor
from data_processing.utils.download_functions import *
os.chdir('esports-data')
os.listdir()

['2021_raw_game_data',
 '2022_raw_game_data',
 '2023_raw_game_data',
 'catboost_info',
 'leagues.json',
 'mapping_data.json',
 'players.json',
 'teams.json',
 'tournaments.json']

In [5]:
# Read in teams data
with open("teams.json", "r") as json_file:
   teams_data = json.load(json_file)

team_df = []
for team in teams_data:
    team_df.append({'team_id': team['team_id'], 'team_name': team['name']})
    
team_df = pd.DataFrame(team_df)

In [6]:
# Read in tournament data
with open("tournaments.json", "r") as json_file:
   tournament_data_all = json.load(json_file)

# Read in mappings data
with open("mapping_data.json", "r") as json_file:
   mappings_data = json.load(json_file)
   
mappings = {
   esports_game["esportsGameId"]: esports_game for esports_game in mappings_data
}

# Set up LeaguesDataProcessor
leagues_data_processor = LeaguesDataProcessor()

In [10]:
tournaments_2021 = [x['slug'] for x in tournament_data_all if '2023' in x['slug']]
# Get rid of international tournaments because they need to be processed separately 
# tournaments_2021.remove('msi_2021')
# tournaments_2021.remove('worlds_2021')

tournaments_2021 = ['pcs_summer_playoffs_2023', 'cblol_2023_split_2', 'lco_split_2_2023', 'lcs_summer_2023', 'lec_summer_2023', 'msi_2023', 'ljl_summer_2023', 'lco_split_1_2023', 'lck_spring_2023', 'arabian_league_summer_2023', 'lck_challengers_summer_2023', 'lck_summer_2023', 'emea_masters_spring_play_ins_2023', 'liga_portuguesa_spring_2023', 'lrn_closing_2023', 'nacl_promotion_spring_2023', 'lec_season_finals_2023', 'pcs_summer_2023', 'hitpoint_masters_summer_2023', 'cblol_academy_2023_split_1', 'nacl_spring_2023', 'liga_master_opening_2023', 'lec_winter_2023', 'nacl_qualifiers_2_spring_2023', 'lcs_spring_2023', 'vdl_opening_2023', 'prime_league_spring_2023', 'gll_spring_2023', 'pcs_spring_playoffs_2023', 'lla_opening_2023', 'pg_nationals_spring_2023', 'cblol_2023_split_1', 'arabian_league_spring_2023']

tournaments = [x for x in tournament_data_all if x['slug'] in tournaments_2021]
tournaments_names = [x['slug'] for x in tournaments]
print(tournaments_names)


['pcs_summer_playoffs_2023', 'cblol_2023_split_2', 'lco_split_2_2023', 'lcs_summer_2023', 'lec_summer_2023', 'msi_2023', 'ljl_summer_2023', 'lco_split_1_2023', 'lck_spring_2023', 'arabian_league_summer_2023', 'lck_challengers_summer_2023', 'lck_summer_2023', 'emea_masters_spring_play_ins_2023', 'liga_portuguesa_spring_2023', 'lrn_closing_2023', 'nacl_promotion_spring_2023', 'lec_season_finals_2023', 'pcs_summer_2023', 'hitpoint_masters_summer_2023', 'cblol_academy_2023_split_1', 'nacl_spring_2023', 'liga_master_opening_2023', 'lec_winter_2023', 'nacl_qualifiers_2_spring_2023', 'lcs_spring_2023', 'vdl_opening_2023', 'prime_league_spring_2023', 'gll_spring_2023', 'pcs_spring_playoffs_2023', 'lla_opening_2023', 'pg_nationals_spring_2023', 'cblol_2023_split_1', 'arabian_league_spring_2023']


In [9]:
# tournaments = [x for x in tournament_data_all if x['slug'] in ['pcs_summer_playoffs_2023']]

In [11]:
"""
Iterate through each tournament and process the data, we will get two dataframes 

tournament_rows = [match_id, esportsGameId, league, team_id_1, team_id_2, outcome_1, outcome_2]
game_rows = [platformGameId, esportsGameId, team_id, start_time, outcome, ...features..., ...league_indicators...]]

Save these dataframes to csv files in the data folder with the corresponding tournament name 
"""

for tournament_data in tournaments:
    tournament_name = tournament_data['slug']
    print("Processing: " + tournament_name)
    # Set up TournamentDataProcessor
    try:
        tournament_data_processor = TournamentDataProcessor(tournament_data, leagues_data_processor.leagues_df)
    except IndexError as e:
        print(f"Error processing tournament {tournament_name}. Tournament could not be found in leagues data")
        continue
    # tournament_data_processor.get_tournament_stages()
    training_data, _ = tournament_data_processor.get_tournament_data(training_stages=[], testing_stages=[])  # Consider all stages to be training 
    print("Games in tournament: " + str(len(training_data)))
    
    # Now download each game, process them, and add them into game_rows 
    directory = "games"
    if not os.path.exists(directory):
      os.makedirs(directory)
    
    # Load each game and process them
    game_rows = []
    for i in tqdm(range(len(training_data))):
        try:
            game_id = training_data.iloc[i]['esportsGameId']
            game_mapping_data = mappings[game_id]
            platform_game_id = game_mapping_data['platformGameId']
            download_gzip_and_write_to_json(f"{directory}/{platform_game_id}")
            with open(f"games/{platform_game_id}.json", "r") as json_file:
                game_data = json.load(json_file)
        except KeyError:
            print(f"Match {game_id} was not found")
            continue
        try:
            game_features = GameFeaturesGenerator(game_data, game_mapping_data).process_game()
        except Exception as e:
            print(f"Error processing game {game_id}")
            print(e)
            continue
        game_features['league'] = training_data.iloc[i]['league']
        game_rows.append(game_features)
    
    game_rows = pd.concat(game_rows)
    game_rows_leagues = leagues_data_processor.transform_league_col(game_rows['league'])
    game_rows = pd.concat([game_rows.drop(['league'],axis=1).reset_index(), game_rows_leagues], axis=1)
    
    # Save to csv in the data folder
    game_data_directory = "2023_raw_game_data"
    if not os.path.exists(game_data_directory):
      os.makedirs(game_data_directory)
    training_data.to_csv(f'{game_data_directory}/{tournament_name}_tournament_rows.csv', index=False)
    game_rows.to_csv(f'{game_data_directory}/{tournament_name}_game_rows.csv', index=False)
    
    # Delete the temp directory
    shutil.rmtree(directory)

Processing: pcs_summer_playoffs_2023
Games in tournament: 42


  0%|          | 0/42 [00:00<?, ?it/s]

File opened
games/ESPORTSTMNT01:3414244.json written
File opened
games/ESPORTSTMNT01:3413275.json written
File opened
games/ESPORTSTMNT01:3415191.json written
File opened
games/ESPORTSTMNT01:3415198.json written
File opened
games/ESPORTSTMNT01:3414077.json written
File opened
games/ESPORTSTMNT01:3414084.json written
File opened
games/ESPORTSTMNT01:3413118.json written
File opened
games/ESPORTSTMNT01:3414497.json written
File opened
games/ESPORTSTMNT01:3413538.json written
File opened
games/ESPORTSTMNT01:3415465.json written
File opened
games/ESPORTSTMNT01:3414360.json written
File opened
games/ESPORTSTMNT01:3414369.json written
File opened
games/ESPORTSTMNT01:3414377.json written
File opened
games/ESPORTSTMNT01:3413797.json written
File opened
games/ESPORTSTMNT01:3415719.json written
File opened
games/ESPORTSTMNT01:3414745.json written
File opened
games/ESPORTSTMNT01:3415800.json written
File opened
games/ESPORTSTMNT01:3413908.json written
File opened
games/ESPORTSTMNT01:3413910.json w

  0%|          | 0/118 [00:00<?, ?it/s]

File opened
games/ESPORTSTMNT01:3376991.json written
File opened
games/ESPORTSTMNT01:3368322.json written
File opened
games/ESPORTSTMNT01:3381541.json written
File opened
games/ESPORTSTMNT03:3177889.json written
File opened
games/ESPORTSTMNT02:3218102.json written
File opened
games/ESPORTSTMNT02:3211413.json written
File opened
games/ESPORTSTMNT02:3218326.json written
File opened
games/ESPORTSTMNT01:3367751.json written
File opened
games/ESPORTSTMNT02:3212243.json written
File opened
games/ESPORTSTMNT02:3212182.json written
File opened
games/ESPORTSTMNT02:3210486.json written
File opened
games/ESPORTSTMNT02:3217355.json written
File opened
games/ESPORTSTMNT01:3367459.json written
File opened
games/ESPORTSTMNT03:3179835.json written
File opened
games/ESPORTSTMNT01:3367768.json written
File opened
games/ESPORTSTMNT01:3380510.json written
File opened
games/ESPORTSTMNT02:3218112.json written
File opened
games/ESPORTSTMNT02:3217378.json written
File opened
games/ESPORTSTMNT01:3380053.json w

...

File opened
games/ESPORTSTMNT03:3098449.json written
File opened
games/ESPORTSTMNT03:3092118.json written
File opened
games/ESPORTSTMNT02:3170395.json written
File opened
games/ESPORTSTMNT03:3098827.json written
File opened
games/ESPORTSTMNT03:3089806.json written
File opened
games/ESPORTSTMNT01:3316440.json written
File opened
games/ESPORTSTMNT02:3169161.json written
File opened
games/ESPORTSTMNT02:3171378.json written
File opened
games/ESPORTSTMNT03:3090682.json written
File opened
games/ESPORTSTMNT01:3316510.json written
File opened
games/ESPORTSTMNT03:3094075.json written
File opened
games/ESPORTSTMNT02:3170083.json written
File opened
games/ESPORTSTMNT03:3100742.json written
File opened
games/ESPORTSTMNT03:3100728.json written
File opened
games/ESPORTSTMNT03:3099366.json written
File opened
games/ESPORTSTMNT02:3169150.json written
File opened
games/ESPORTSTMNT03:3091672.json written
File opened
games/ESPORTSTMNT01:3316459.json written
File opened
games/ESPORTSTMNT02:3170118.json w

  0%|          | 0/120 [00:00<?, ?it/s]

File opened
games/ESPORTSTMNT01:3300699.json written
File opened
games/ESPORTSTMNT01:3313248.json written
File opened
games/ESPORTSTMNT03:3094757.json written
File opened
games/ESPORTSTMNT03:3086706.json written
File opened
games/ESPORTSTMNT03:3087532.json written
File opened
games/ESPORTSTMNT03:3092417.json written
File opened
games/ESPORTSTMNT01:3302258.json written
File opened
games/ESPORTSTMNT02:3153877.json written
File opened
games/ESPORTSTMNT01:3311651.json written
File opened
games/ESPORTSTMNT03:3087692.json written
File opened
games/ESPORTSTMNT01:3302316.json written
File opened
games/ESPORTSTMNT03:3093417.json written
File opened
games/ESPORTSTMNT02:3153825.json written
File opened
games/ESPORTSTMNT03:3092815.json written
File opened
games/ESPORTSTMNT01:3309743.json written
File opened
games/ESPORTSTMNT01:3313202.json written
File opened
games/ESPORTSTMNT03:3088580.json written
File opened
games/ESPORTSTMNT03:3092442.json written
File opened
games/ESPORTSTMNT03:3092740.json w

  0%|          | 0/70 [00:00<?, ?it/s]

File opened
games/ESPORTSTMNT03:3098830.json written
File opened
games/ESPORTSTMNT01:3306704.json written
File opened
games/ESPORTSTMNT01:3307039.json written
File opened
games/ESPORTSTMNT01:3310290.json written
File opened
games/ESPORTSTMNT01:3316205.json written
File opened
games/ESPORTSTMNT03:3100388.json written
File opened
games/ESPORTSTMNT03:3096856.json written
File opened
games/ESPORTSTMNT01:3317175.json written
File opened
games/ESPORTSTMNT01:3306621.json written
File opened
games/ESPORTSTMNT03:3100359.json written
File opened
games/ESPORTSTMNT01:3307101.json written
File opened
games/ESPORTSTMNT03:3097984.json written
File opened
games/ESPORTSTMNT01:3310160.json written
File opened
games/ESPORTSTMNT01:3311155.json written
File opened
games/ESPORTSTMNT03:3098049.json written
File opened
games/ESPORTSTMNT03:3098400.json written
File opened
games/ESPORTSTMNT01:3306190.json written
File opened
games/ESPORTSTMNT03:3098787.json written
File opened
games/ESPORTSTMNT01:3315287.json w

In [14]:
 tournament_data.keys()

dict_keys(['id', 'leagueId', 'name', 'slug', 'sport', 'startDate', 'endDate', 'stages'])

In [6]:
leagues_data_processor.leagues_df

,league_id,league_name,league_region,league_tournaments
0,98767991299243165,LCS,NORTH AMERICA,110303581083678395
1,98767991299243165,LCS,NORTH AMERICA,109517090066605615
2,98767991299243165,LCS,NORTH AMERICA,108206581962155974
3,98767991299243165,LCS,NORTH AMERICA,107458367237283414
4,98767991299243165,LCS,NORTH AMERICA,107458335260330212
...,...,...,...,...
5,105549980953490846,CBLOL Academy,BRAZIL,105562692792011783
0,110371976858004491,North Regional League,LATIN AMERICA NORTH,110372140231846916
0,110372322609949919,South Regional League,LATIN AMERICA SOUTH,110372454557117766
0,108001239847565215,TFT Rising Legends,INTERNATIONAL,109761195185432372


In [7]:
[x for x in leagues_data_processor.leagues_df['league_id'] if x=='101097443346691685']

[]

In [9]:
np.sort(leagues_data_processor.leagues_df['league_id'])

array(['100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '101382741235120470',
       '101382741235120470', '101382741235120470', '101382741235120470',
       '101382741235120470', '101382741235120470', '101382741235120470',
       '101382741235120470', '101382741235120470', '101382741235120470',
       '101382741235120470', '101382741235120470', '104366947889790212',
       '104366947889790212', '104366947889790212', '104366947889790212',
       '104366947889790212', '104366947889790212', '104366947889790212',
       '104366947889790212', '104366947889790212', '104366947889790212',
       '105266074488398661', '105266074488398661', '105266074488398661',
       '105266074488398661', '105266074488398661', 